In [26]:
import pandas as pd
import datetime as dt
import numpy as np
import requests
import tqdm
import pickle
import warnings
import tensorflow as tf
warnings.filterwarnings('ignore')

In [27]:
def make_params(starttime, endtime, minlatitude, maxlatitude, minlongitude, maxlongitude):
    params = {
        "format": "geojson",
        "starttime": starttime,
        "endtime": endtime,
        "minlatitude": minlatitude,
        "maxlatitude": maxlatitude,
        "minlongitude": minlongitude,
        "maxlongitude": maxlongitude
    }
    return params

In [28]:
def make_params_circle(starttime, endtime, latitude, longitude, maxradiuskm):
    params = {
        "format": "geojson",
        "starttime": starttime,
        "endtime": endtime,
        "latitude": latitude,
        "longitude": longitude,
        "maxradiuskm": maxradiuskm
    }
    return params

In [58]:
def get_earthquake_count(params):
    url = "https://earthquake.usgs.gov/fdsnws/event/1/count"
    response = requests.get(url, params=params)
    print("XD")
    print(response)
    return response.json()["count"]

In [56]:
def get_earthquake_data(params):
    url = "https://earthquake.usgs.gov/fdsnws/event/1/query"
    response = requests.get(url, params=params)
    print(response)
    return response

In [31]:
def make_df(resp, params, errors):
    all_eqs = []
    try:
        for eq in resp.json()["features"]:
            prop = list(eq["properties"].values())
            prop.extend(eq["geometry"]["coordinates"])
            all_eqs.append(prop)
        cols = list(resp.json()["features"][0]["properties"].keys())
        cols.extend(["longitude", "latitude", "depth"])
        df = pd.DataFrame(all_eqs, columns=cols)
    except:
        errors.append(params)
        df = pd.DataFrame()
    return df, errors

In [32]:
def haversine_distance(lat1, lon1, lat2, lon2):
    R = 6371.0
    lat1 = np.radians(lat1)
    lon1 = np.radians(lon1)
    lat2 = np.radians(lat2)
    lon2 = np.radians(lon2)
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    distance = R * c
    return distance

In [34]:
def find_min_dist(df_tp, x, y):
    df_tp["dist"] = haversine_distance(x, y, df_tp["lat"], df_tp["lon"])
    min_dist = df_tp["dist"].min()
    plates = df_tp[df_tp["dist"] == min_dist].sort_values("plate")["plate"].tolist()
    plates = "_".join(plates)
    return min_dist, plates

In [35]:
def add_region_info(df, df_tp, geo_split):
    region2plate = {}
    region2dist = {}
    df["lat_cent"] = df["latitude_disc"] + 0.5
    df["lon_cent"] = df["longitude_disc"] + 0.5
    for pos in tqdm.tqdm(df["pos"].unique()):
        x, y = pos.split("_")
        x, y = float(x), float(y)
        dist, plate = find_min_dist(df_tp, x + 0.5, y + 0.5)
        region2plate[pos] = plate
        region2dist[pos] = dist
    df["plate_region"] = df["pos"].map(region2plate)
    df["dist_region"] = df["pos"].map(region2dist)
    return df

In [36]:
def add_tectonic_info(df, df_tp):
    coordinates = list(zip(df['latitude'], df['longitude']))
    results = list(tqdm.tqdm(map(lambda x: find_min_dist(df_tp, x[0], x[1]), coordinates), total=len(coordinates)))
    df[["dist", "plate"]] = results
    return df

In [37]:
def map_col(df, col, mapping):
    mapping = dict(zip(mapping.iloc[:, 0], mapping.iloc[:, 1]))
    df[col] = df[col].map(mapping)
    return df

In [38]:
PREPROC_PARAMS = {
    "mag_low": -1,
    "mag_high": 7,
    "depth_low": 2,
    "depth_high": 1e8,
    "dist_low": 1,
    "dist_high": 1e8,
    "dist_region_low": 2,
    "dist_region_high": 1e8,
    "scale_distance": 78.44,
    "scale_distance_lag": 300,
}

In [39]:
def preprocess_df(df, PREPROC_PARAMS, scaler_dict):

    scaler = scaler_dict["mag"]
    df["mag"] = scaler.transform(np.clip(df["mag"].values, PREPROC_PARAMS["mag_low"], PREPROC_PARAMS["mag_high"]).reshape(-1, 1))

    scaler = scaler_dict["depth"]
    df["depth"] = np.log(df["depth"] + np.abs(df["depth"].min()) + 1)
    df["depth"] = scaler.transform(np.clip(df["depth"].values, PREPROC_PARAMS["depth_low"], PREPROC_PARAMS["depth_high"]).reshape(-1, 1))

    scaler = scaler_dict["latitude_new"]
    df["latitude_new"] = scaler.transform(df["latitude"].values.reshape(-1, 1))

    scaler = scaler_dict["longitude_new"]
    df["longitude_new"] = scaler.transform(df["longitude"].values.reshape(-1, 1))

    scaler = scaler_dict["lat_cent"]
    df["lat_cent"] = scaler.transform(df["lat_cent"].values.reshape(-1, 1))

    scaler = scaler_dict["lon_cent"]
    df["lon_cent"] = scaler.transform(df["lon_cent"].values.reshape(-1, 1))

    scaler = scaler_dict["dist"]
    df["dist"] = df["dist"].astype(float)
    df["dist"] = scaler.transform(np.clip(np.log(df["dist"] + 1).values.reshape(-1, 1), PREPROC_PARAMS["dist_low"], PREPROC_PARAMS["dist_high"]))

    scaler = scaler_dict["dist_region"]
    df["dist_region"] = scaler.transform(np.clip(np.log(df["dist_region"] + 1).values.reshape(-1, 1), PREPROC_PARAMS["dist_region_low"], PREPROC_PARAMS["dist_region_high"]))

    return df

In [40]:
def make_block(df, pos, radius, block_size, PREPROC_PARAMS):
    bins = [0, 1, 2, 3, 4, 5, 6, 7, 10, 14, 21, 30, 60, 180, 1e8]
    lat, lon = pos.split("_")
    lat, lon = float(lat), float(lon)
    tmp1 = df[df["pos"] == pos]
    diff = int(radius / 111) + 3
    tmp2 = df[((df["latitude"] >= lat - diff) & (df["latitude"] <= lat + diff) & (df["longitude"] >= lon - diff) & (df["longitude"] <= lon + diff)) & (df["pos"] != pos)]
    tmp1["label"] = 0
    tmp2["label"] = -1
    tmp = pd.concat([tmp1, tmp2], axis=0)
    tmp = tmp[tmp["distance"] <= radius]
    tmp.sort_values(by=["time"], inplace=True)
    tmp["diff_days"] = (tmp["time"] - tmp["time"].shift(1)).dt.days
    tmp.dropna(inplace=True)
    tmp["diff_days"] = np.digitize(tmp["diff_days"], bins=bins) - 1
    for idx in range(1, block_size):
        tmp["mag_" + str(idx)] = tmp["mag"].shift(idx)
        tmp["depth_" + str(idx)] = tmp["depth"].shift(idx)
        tmp["latitude_new_" + str(idx)] = tmp["latitude_new"].shift(idx)
        tmp["longitude_new_" + str(idx)] = tmp["longitude_new"].shift(idx)
        tmp["dist_" + str(idx)] = tmp["dist"].shift(idx)
        tmp["distance_" + str(idx)] = tmp["distance"].shift(idx) / PREPROC_PARAMS["scale_distance_lag"]
        tmp["plate_" + str(idx)] = tmp["plate"].shift(idx)
        tmp["diff_days_" + str(idx)] = tmp["diff_days"].shift(idx)
        tmp["magType_" + str(idx)] = tmp["magType"].shift(idx)
    tmp = tmp[tmp["label"] != -1]
    tmp["distance"] = tmp["distance"] / PREPROC_PARAMS["scale_distance"]
    tmp.dropna(inplace=True)
    return tmp

In [41]:
def reshape(df, block_size, feature_order, featrues_region):
    x_ts = df[feature_order].to_numpy().reshape(-1, block_size, len(feature_order) // block_size)
    x_region = df[featrues_region].to_numpy().reshape(-1, len(featrues_region))
    return x_ts, x_region

In [42]:
def check_coords(X, Y, regions):
    if X < -180 or X > 180 or Y < -90 or Y > 90:
        return False
    for pos in regions:
        y, x = pos.split("_")
        y, x = float(y), float(x)
        if X >= x and X <= x + GEO_SPLIT and Y >= y and Y <= y + GEO_SPLIT:
            return True
    return False

In [43]:
scalers = pickle.load(open("../data/scalers_for_npys.pkl", "rb"))
regions = np.load("../data/regions.npy", allow_pickle=True)

In [44]:
from typing import Dict, Union
from sklearn.preprocessing import MinMaxScaler
def make_timeseries(
    df: pd.DataFrame,
    x: float,
    y: float,
    radius: int,
    block_size: int,
    features_order: List[str],
    features_region: List[str],
    preproc_params: Dict[str, Union[int, float, List[int]]],
    scaler_dict: Dict[str, MinMaxScaler],
) -> Tuple[np.ndarray, np.ndarray]:
    df["time"] = pd.to_datetime(df["time"], format="mixed")
    df.sort_values(by="time", inplace=True)
    pos = str(y - GEO_SPLIT / 2) + "_" + str(x - GEO_SPLIT / 2)
    df = preprocess_df(df, preproc_params, scaler_dict)
    df_pos = make_block(df, pos, radius, block_size, preproc_params)
    df_pos = df_pos.iloc[-1]
    time = df_pos["time"]
    if dt.datetime.now() - time > dt.timedelta(days=30):
        return None, None, None
    x_ts, x_region = reshape(df_pos, block_size, features_order, features_region)
    return x_ts, x_region, time

In [45]:
def prepare_data(df: pd.DataFrame, geo_split: int) -> pd.DataFrame:
    df_tp = pd.read_csv("../data/all.csv")
    df_tp.drop_duplicates(inplace=True)

    df = add_region_info(df, df_tp, geo_split)
    df = add_tectonic_info(df, df_tp)
    mapping1 = pd.read_csv("../data/magtype2id.csv")
    mapping2 = pd.read_csv("../data/plate2id.csv")
    mapping3 = pd.read_csv("../data/plate_region2id.csv")

    df = map_col(df, "magType", mapping1)
    df = map_col(df, "plate", mapping2)
    df = map_col(df, "plate_region", mapping3)

    df["plate_region"] = df["plate_region"].fillna(51)
    df["plate"] = df["plate"].fillna(61)
    return df

In [46]:
def make_feature_order(features: List[str], block_size: int) -> List[str]:
    """
    Creates the feature order for the time-series block. The order is defined as follows: [feature_1_lag_block_size-1, feature_2_lag_block_size-1, ..., feature_1_lag_0, feature_2_lag_0].

    Parameters:
    - features (List[str]): The features of the block.
    - block_size (int): The size of the block.

    Returns:
    - features_order (List[str]): The feature order of the block.

    Example:
    >>> make_feature_order(["mag", "depth"], 2)
    ['mag_1', 'depth_1', 'mag_0', 'depth_0']
    """
    features_order = [
        features[idx] + "_" + str(i)
        for i in range(block_size - 1, 0, -1)
        for idx in range(len(features))
    ]
    features_order = features_order + features
    return features_order

In [47]:
FEATURES_REGION: List[str] = ["lat_cent", "lon_cent", "dist_region", "plate_region"]
FEATURES: List[str] = [
    "mag",
    "depth",
    "latitude_new",
    "longitude_new",
    "dist",
    "distance",
    "plate",
    "diff_days",
    "magType",
]

In [60]:
def get_data(X, Y, start_time, end_time, min_lat, max_lat, min_lon, max_lon, geo_split, radius, block_size):
    errors = []
    count = get_earthquake_count(
        make_params(start_time, end_time, min_lat, max_lat, min_lon, max_lon)
    )
    if count == 0:
        print("Last earthquake was more than 30 days ago")
        return None, None, None
    print(start_time, count)
    resp = get_earthquake_data(
        make_params(start_time, end_time, min_lat, max_lat, min_lon, max_lon)
    )
    df, errors = make_df(
        resp,
        make_params(start_time, end_time, min_lat, max_lat, min_lon, max_lon),
        errors,
    )
    df = df[df["type"] == "earthquake"]
    df["time"] = df["time"].apply(lambda x: dt.datetime.fromtimestamp(x / 1000))
    df["time"] = pd.to_datetime(df["time"], format="mixed")
    df = df[["time", "longitude", "latitude", "depth", "mag", "magType"]]
    df.dropna(inplace=True)
    df.drop_duplicates(inplace=True)
    if len(df) == 0:
        print("Last earthquake was more than 30 days ago v2")
        return None, None, None # No data
    end_time = df["time"].max()
    count = 0
    cnt = 0
    while count < 65:
        if cnt > 50:
            print("No data")
            return None, None, None
        start_time = start_time - dt.timedelta(days=360)
        count = get_earthquake_count(
            make_params_circle(start_time, end_time, Y, X, radius+10)
        )
        cnt += 1
        print(start_time, end_time, count)

    df = pd.DataFrame()
    while cnt < 20:
        cnt += 1
        n = count // 20000
        dates = pd.date_range(start_time, end_time, periods= 2 if count < 20000 else n + 4)
        for i in range(len(dates)-1):
            print(dates[i], dates[i+1])
            resp = get_earthquake_data(
                    make_params_circle(
                        dates[i].to_pydatetime(),
                        dates[i + 1].to_pydatetime(),
                        Y,
                        X,
                        radius+10,
                    )
                )
            df_small, errors = make_df(
                resp,
                make_params_circle(
                    dates[i].to_pydatetime(),
                    dates[i + 1].to_pydatetime(),
                    Y,
                    X,
                    radius+10,
                ),
                errors,
            )
            df_small = df_small[df_small["type"] == "earthquake"]
            df_small = df_small[
                ["time", "longitude", "latitude", "depth", "mag", "magType"]
            ]
            df_small["time"] = df_small["time"].apply(
                lambda x: dt.datetime.fromtimestamp(x / 1000)
            )
            df_small.dropna(inplace=True)
            df_small.drop_duplicates(inplace=True)
            df = pd.concat([df, df_small], axis=0).reset_index(drop=True)
        df["pos"] = "0_0"
        df.loc[
            (df["latitude"] >= Y - geo_split / 2)
            & (df["latitude"] <= Y + geo_split / 2)
            & (df["longitude"] >= X - geo_split / 2)
            & (df["longitude"] <= X + geo_split / 2),
            "pos",
        ] = (
            str(Y - geo_split / 2) + "_" + str(X - geo_split / 2)
        )
        df["distance"] = haversine_distance(df["latitude"], df["longitude"], Y, X)
        df["latitude_disc"] = Y - geo_split / 2
        df["longitude_disc"] = X - geo_split / 2
        tmp = df[df["distance"] <= radius]
        if len(tmp) >= block_size + 1:
            return (tmp,
                start_time,
                errors,
            )
        end_time = start_time
        start_time = start_time - dt.timedelta(days=360)
    print("No data")
    return None, None, None

In [61]:
now = dt.datetime.now()
START_TIME = now - dt.timedelta(days=30)
END_TIME = now
GEO_SPLIT = 1
RADIUS = 300
BLOCK_SIZE = 64
cnt = 0
# for i in range(100_000):
#     X = 16
#     Y = 44
#     MIN_LAT = Y - GEO_SPLIT / 2
#     MAX_LAT = Y + GEO_SPLIT / 2
#     MIN_LON = X - GEO_SPLIT / 2
#     MAX_LON = X + GEO_SPLIT / 2
#     try:
#         df, start_time, errors = get_data(X, Y, START_TIME, END_TIME, MIN_LAT, MAX_LAT, MIN_LON, MAX_LON, GEO_SPLIT, RADIUS, BLOCK_SIZE)
#         if df is not None:
#             print(len(df))
#             cnt += 1
#             print("-" * 20)
#     except:
#         print(X, Y, START_TIME, END_TIME, MIN_LAT, MAX_LAT, MIN_LON, MAX_LON, GEO_SPLIT, RADIUS, BLOCK_SIZE)

In [62]:
X = -149
Y = 60
MIN_LAT = Y - GEO_SPLIT / 2
MAX_LAT = Y + GEO_SPLIT / 2
MIN_LON = X - GEO_SPLIT / 2
MAX_LON = X + GEO_SPLIT / 2
dates = get_data(X, Y, START_TIME, END_TIME, MIN_LAT, MAX_LAT, MIN_LON, MAX_LON, GEO_SPLIT, RADIUS, BLOCK_SIZE)

XD
<Response [200]>
2023-12-25 02:18:36.457885 3
<Response [200]>
XD
<Response [200]>
2022-12-30 02:18:36.457885 2024-01-01 18:14:49.054000 19488
